In [14]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score

In [13]:
dataset0 = pd.read_csv('data/dataset0.csv')
dataset0.label.replace(-1,0,inplace=True)
dataset1 = pd.read_csv('data/dataset1.csv')
dataset1.label.replace(-1,0,inplace=True)
dataset2 = pd.read_csv('data/dataset2.csv')
dataset2.label.replace(-1,0,inplace=True)
dataset3 = pd.read_csv('data/dataset3.csv')

dataset0.drop_duplicates(inplace=True)
dataset1.drop_duplicates(inplace=True)
dataset2.drop_duplicates(inplace=True)
dataset3.drop_duplicates(inplace=True)
print(dataset3.info())

dataset01 = pd.concat([dataset0,dataset1],axis=0)
dataset012 = pd.concat([dataset01,dataset2],axis=0)

dataset01_y = dataset01.label
dataset01_x = dataset01.drop(['user_id','label'],axis=1)  # 'day_gap_before','day_gap_after' cause overfitting, 0.77
dataset2_y = dataset2.label
dataset2_x = dataset2.drop(['user_id','label'],axis=1)
dataset012_y = dataset012.label
dataset012_x = dataset012.drop(['user_id','label'],axis=1)
dataset3_preds = dataset3[['user_id','coupon_id','date_received']]
dataset3_x = dataset3.drop(['user_id','coupon_id','date_received'],axis=1)

print(dataset01_x.shape,dataset2_x.shape,dataset3_x.shape)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124056 entries, 0 to 124055
Data columns (total 50 columns):
user_id                                         124056 non-null int64
coupon_id                                       124056 non-null int64
discount_rate                                   124056 non-null float64
distance                                        111117 non-null float64
date_received                                   124056 non-null int64
day_of_month                                    124056 non-null int64
days_distance                                   124056 non-null int64
discount_man                                    121382 non-null float64
discount_jian                                   121382 non-null float64
is_man_jian                                     124056 non-null int64
total_sales                                     123995 non-null float64
sales_use_coupon                                124047 non-null float64
total_coupon                         

In [17]:
param_test = {}


params={'booster':'gbtree',
	    'gamma':0.1,
	    'min_child_weight':1.1,
	    'max_depth':5,
	    'lambda':10,
	    'subsample':0.65,
	    'colsample_bytree':0.65,
	    'colsample_bylevel':0.65,
	    'eta': 0.01,
	    'tree_method':'exact',
	    'seed':0,
	    'nthread':12
	    }

In [18]:
dataset01 = xgb.DMatrix(dataset01_x,label=dataset01_y)
dataset2 = xgb.DMatrix(dataset2_x,label=dataset2_y)
dataset012 = xgb.DMatrix(dataset012_x,label=dataset012_y)
dataset3 = xgb.DMatrix(dataset3_x)

In [ ]:
watchlist = [(dataset01,'train')]
# train on dataset01, evaluate on dataset2
model = xgb.train(params,dataset01,num_boost_round=2000,evals=watchlist)
preds = model.predict(dataset2)
preds = MinMaxScaler().fit_transform(preds.reshape(-1,1))
print(roc_auc_score(dataset2.get_label(),preds))
# for submit
watchlist = [(dataset012,'train')]
model = xgb.train(params,dataset012,num_boost_round=2000,evals=watchlist)
#predict test set
dataset3_preds['label'] = model.predict(dataset3)
dataset3_preds.label = MinMaxScaler().fit_transform(dataset3_preds.label.reshape(-1,1))
dataset3_preds.sort_values(by=['coupon_id','label'],inplace=True)
dataset3_preds.to_csv("xgb_preds.csv",index=None,header=None)

[0]	train-rmse:0.496291
[1]	train-rmse:0.492645
[2]	train-rmse:0.489048
[3]	train-rmse:0.485501
[4]	train-rmse:0.48199
[5]	train-rmse:0.478549
[6]	train-rmse:0.475142
[7]	train-rmse:0.471739
[8]	train-rmse:0.468402
[9]	train-rmse:0.465098
[10]	train-rmse:0.461848
[11]	train-rmse:0.458641
[12]	train-rmse:0.455468
[13]	train-rmse:0.452327
[14]	train-rmse:0.449222
[15]	train-rmse:0.446162
[16]	train-rmse:0.443154
[17]	train-rmse:0.440185
[18]	train-rmse:0.437235
[19]	train-rmse:0.434367
[20]	train-rmse:0.431499
[21]	train-rmse:0.428699
[22]	train-rmse:0.425908
[23]	train-rmse:0.423144
[24]	train-rmse:0.420442
[25]	train-rmse:0.417779
[26]	train-rmse:0.415138
[27]	train-rmse:0.412536
[28]	train-rmse:0.409968
[29]	train-rmse:0.407447
[30]	train-rmse:0.404976
[31]	train-rmse:0.402513
[32]	train-rmse:0.400093
[33]	train-rmse:0.397703
[34]	train-rmse:0.395347
[35]	train-rmse:0.393033
[36]	train-rmse:0.390745
[37]	train-rmse:0.388501
[38]	train-rmse:0.386268
[39]	train-rmse:0.384074
[40]	train-

[321]	train-rmse:0.245663
[322]	train-rmse:0.245633
[323]	train-rmse:0.245606
[324]	train-rmse:0.24558
[325]	train-rmse:0.245553
[326]	train-rmse:0.245524
[327]	train-rmse:0.245493
[328]	train-rmse:0.245464
[329]	train-rmse:0.245435
[330]	train-rmse:0.245402
[331]	train-rmse:0.245378
[332]	train-rmse:0.245349
[333]	train-rmse:0.245318
[334]	train-rmse:0.245299
[335]	train-rmse:0.245269
[336]	train-rmse:0.245238
[337]	train-rmse:0.245208
[338]	train-rmse:0.245181
[339]	train-rmse:0.245154
[340]	train-rmse:0.245129
[341]	train-rmse:0.245103
[342]	train-rmse:0.245073
[343]	train-rmse:0.24505
[344]	train-rmse:0.24503
[345]	train-rmse:0.245006
[346]	train-rmse:0.244985
[347]	train-rmse:0.244965
[348]	train-rmse:0.244946
[349]	train-rmse:0.244921
[350]	train-rmse:0.244894
[351]	train-rmse:0.244856
[352]	train-rmse:0.244836
[353]	train-rmse:0.244814
[354]	train-rmse:0.244789
[355]	train-rmse:0.244765
[356]	train-rmse:0.244739
[357]	train-rmse:0.244718
[358]	train-rmse:0.244701
[359]	train-rms

[638]	train-rmse:0.241046
[639]	train-rmse:0.24104
[640]	train-rmse:0.241028
[641]	train-rmse:0.241016
[642]	train-rmse:0.241004
[643]	train-rmse:0.240992
[644]	train-rmse:0.240989
[645]	train-rmse:0.240982
[646]	train-rmse:0.240978
[647]	train-rmse:0.240968
[648]	train-rmse:0.240962
[649]	train-rmse:0.240955
[650]	train-rmse:0.240948
[651]	train-rmse:0.240938
[652]	train-rmse:0.240933
[653]	train-rmse:0.240928
[654]	train-rmse:0.240919
[655]	train-rmse:0.240909
[656]	train-rmse:0.240899
[657]	train-rmse:0.240893
[658]	train-rmse:0.240884
[659]	train-rmse:0.240871
[660]	train-rmse:0.240854
[661]	train-rmse:0.240841
[662]	train-rmse:0.240836
[663]	train-rmse:0.24083
[664]	train-rmse:0.240819
[665]	train-rmse:0.240815
[666]	train-rmse:0.240812
[667]	train-rmse:0.240806
[668]	train-rmse:0.240796
[669]	train-rmse:0.240787
[670]	train-rmse:0.240777
[671]	train-rmse:0.240767
[672]	train-rmse:0.240759
[673]	train-rmse:0.240747
[674]	train-rmse:0.240741
[675]	train-rmse:0.240729
[676]	train-rm